# Lab 01: Analyze Images with Azure AI Vision

## Overview
Azure AI Vision is an artificial intelligence capability that enables software systems to interpret visual input by analyzing images. In this lab, you'll use the Azure AI Vision service to:
- Generate captions and tags for images
- Detect common objects in images
- Detect people in images
- Remove background or create foreground matting

## Prerequisites
- An Azure subscription with an Azure AI Vision resource
- The endpoint and key for your Azure AI Vision resource

## Learning Objectives
By the end of this lab, you will be able to:
- Authenticate with Azure AI Vision service
- Analyze images to extract visual features
- Detect and annotate objects and people in images
- Visualize detection results

## Step 1: Install Required Packages

First, let's install the necessary Python packages for working with Azure AI Vision.

In [ ]:
# Install required packages
# Note: Uncomment the following line if running in a fresh environment
# !pip install python-dotenv azure-ai-vision-imageanalysis==1.0.0 matplotlib pillow

## Step 2: Import Required Libraries

Import all the necessary Python libraries for image analysis.

In [ ]:
from dotenv import load_dotenv
import os
from PIL import Image, ImageDraw
import sys
from matplotlib import pyplot as plt
from azure.core.exceptions import HttpResponseError
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

# Enable inline plotting for Jupyter notebooks
%matplotlib inline

print("✓ Libraries imported successfully")

## Step 3: Configure Azure AI Vision Credentials

Load your Azure AI Vision endpoint and key from environment variables. You can either:
1. Create a `.env` file in the `python/image-analysis` directory with your credentials
2. Set the values directly in the cells below

In [ ]:
# Load configuration from .env file
load_dotenv('python/image-analysis/.env')

# Get Configuration Settings
ai_endpoint = os.getenv('AI_SERVICE_ENDPOINT')
ai_key = os.getenv('AI_SERVICE_KEY')

# Alternatively, set values directly (NOT recommended for production)
# ai_endpoint = 'your_azure_ai_services_endpoint'
# ai_key = 'your_azure_ai_services_key'

# Validate credentials are loaded
if not ai_endpoint or not ai_key or 'your_' in ai_endpoint or 'your_' in ai_key:
    print("⚠ Warning: Please configure your Azure AI Vision credentials")
    print("  Update the .env file or set the values directly in this cell")
else:
    print(f"✓ Endpoint configured: {ai_endpoint[:30]}...")
    print("✓ API key loaded")

## Step 4: Authenticate Azure AI Vision Client

Create an authenticated client to interact with the Azure AI Vision service.

In [ ]:
# Authenticate Azure AI Vision client
cv_client = ImageAnalysisClient(
    endpoint=ai_endpoint,
    credential=AzureKeyCredential(ai_key)
)

print("✓ Azure AI Vision client authenticated successfully")

## Step 5: Analyze an Image

Let's analyze an image to extract captions, tags, objects, and people. We'll use a street scene image as an example.

In [ ]:
# Specify the image to analyze
image_file = 'python/image-analysis/images/street.jpg'

# Display the image
print(f"Analyzing image: {image_file}")
img = Image.open(image_file)
plt.figure(figsize=(10, 8))
plt.imshow(img)
plt.axis('off')
plt.title('Original Image')
plt.show()

print(f"Image size: {img.size[0]}x{img.size[1]} pixels")

### 5.1 Get Image Captions

The Azure AI Vision service can generate descriptive captions for images using advanced AI models.

In [ ]:
# Get image analysis with caption feature
with open(image_file, "rb") as f:
    image_data = f.read()

result = cv_client.analyze(
    image_data=image_data,
    visual_features=[VisualFeatures.CAPTION]
)

# Display the caption
if result.caption is not None:
    print("\n=== Image Caption ===")
    print(f"Caption: '{result.caption.text}'")
    print(f"Confidence: {result.caption.confidence:.2%}")

### 5.2 Get Image Tags

Tags provide descriptive keywords about the content of the image.

In [ ]:
# Get image tags
result = cv_client.analyze(
    image_data=image_data,
    visual_features=[VisualFeatures.TAGS]
)

# Display tags
if result.tags is not None:
    print("\n=== Image Tags ===")
    print(f"Found {len(result.tags.list)} tags:")
    for tag in result.tags.list:
        print(f"  • {tag.name} (confidence: {tag.confidence:.2%})")

### 5.3 Detect Objects

The service can identify common objects in images and provide their locations using bounding boxes.

In [ ]:
# Get objects in the image
result = cv_client.analyze(
    image_data=image_data,
    visual_features=[VisualFeatures.OBJECTS]
)

# Display detected objects
if result.objects is not None:
    print("\n=== Detected Objects ===")
    print(f"Found {len(result.objects.list)} objects:")
    for obj in result.objects.list:
        print(f"  • {obj.tags[0].name} (confidence: {obj.tags[0].confidence:.2%})")
        r = obj.bounding_box
        print(f"    Location: ({r.x}, {r.y}), Size: {r.width}x{r.height}")

### 5.4 Visualize Object Detection

Let's draw bounding boxes around the detected objects to visualize the results.

In [ ]:
def annotate_objects(image_filename, detected_objects):
    """
    Draw bounding boxes around detected objects in an image.
    
    Args:
        image_filename: Path to the image file
        detected_objects: List of detected objects from Azure AI Vision
    """
    print("\nAnnotating objects...")
    
    # Prepare image for drawing
    image = Image.open(image_filename)
    fig = plt.figure(figsize=(image.width/100, image.height/100))
    plt.axis('off')
    draw = ImageDraw.Draw(image)
    color = 'cyan'
    
    for detected_object in detected_objects:
        # Draw object bounding box
        r = detected_object.bounding_box
        bounding_box = ((r.x, r.y), (r.x + r.width, r.y + r.height)) 
        draw.rectangle(bounding_box, outline=color, width=3)
        plt.annotate(detected_object.tags[0].name, (r.x, r.y), backgroundcolor=color)
    
    # Display annotated image
    plt.imshow(image)
    plt.tight_layout(pad=0)
    plt.title('Detected Objects')
    plt.show()
    print("✓ Object annotation complete")

# Annotate the image with detected objects
if result.objects is not None and len(result.objects.list) > 0:
    annotate_objects(image_file, result.objects.list)
else:
    print("No objects detected to annotate")

### 5.5 Detect People

The Azure AI Vision service can also detect people in images and provide bounding boxes around them.

In [ ]:
# Get people in the image
result = cv_client.analyze(
    image_data=image_data,
    visual_features=[VisualFeatures.PEOPLE]
)

# Display detected people
if result.people is not None:
    print("\n=== Detected People ===")
    print(f"Found {len(result.people.list)} people:")
    for person in result.people.list:
        if person.confidence > 0.5:
            r = person.bounding_box
            print(f"  • Person (confidence: {person.confidence:.2%})")
            print(f"    Location: ({r.x}, {r.y}), Size: {r.width}x{r.height}")

### 5.6 Visualize People Detection

Let's draw bounding boxes around the detected people.

In [ ]:
def annotate_people(image_filename, detected_people):
    """
    Draw bounding boxes around detected people in an image.
    
    Args:
        image_filename: Path to the image file
        detected_people: List of detected people from Azure AI Vision
    """
    print("\nAnnotating people...")
    
    # Prepare image for drawing
    image = Image.open(image_filename)
    fig = plt.figure(figsize=(image.width/100, image.height/100))
    plt.axis('off')
    draw = ImageDraw.Draw(image)
    color = 'cyan'
    
    for detected_person in detected_people:
        if detected_person.confidence > 0.5:
            # Draw bounding box
            r = detected_person.bounding_box
            bounding_box = ((r.x, r.y), (r.x + r.width, r.y + r.height))
            draw.rectangle(bounding_box, outline=color, width=3)
    
    # Display annotated image
    plt.imshow(image)
    plt.tight_layout(pad=0)
    plt.title('Detected People')
    plt.show()
    print("✓ People annotation complete")

# Annotate the image with detected people
if result.people is not None and len(result.people.list) > 0:
    annotate_people(image_file, result.people.list)
else:
    print("No people detected to annotate")

## Step 6: Try Different Images

Now let's analyze other images in the dataset. You can modify the `image_file` variable to analyze different images.

In [ ]:
# List available images
import glob

image_dir = 'python/image-analysis/images'
available_images = glob.glob(f"{image_dir}/*.jpg")

print("Available images for analysis:")
for idx, img_path in enumerate(available_images, 1):
    print(f"{idx}. {os.path.basename(img_path)}")

In [ ]:
# Analyze a different image (e.g., person.jpg or building.jpg)
# Uncomment and run to analyze another image:

# image_file = 'python/image-analysis/images/person.jpg'
# # or
# image_file = 'python/image-analysis/images/building.jpg'

# # Display the image
# img = Image.open(image_file)
# plt.figure(figsize=(10, 8))
# plt.imshow(img)
# plt.axis('off')
# plt.title(f'Analyzing: {os.path.basename(image_file)}')
# plt.show()

# # Get comprehensive analysis
# with open(image_file, "rb") as f:
#     image_data = f.read()

# result = cv_client.analyze(
#     image_data=image_data,
#     visual_features=[VisualFeatures.CAPTION, VisualFeatures.TAGS, 
#                      VisualFeatures.OBJECTS, VisualFeatures.PEOPLE]
# )

# # Display all results
# if result.caption:
#     print(f"\nCaption: {result.caption.text} ({result.caption.confidence:.2%})")
# if result.tags:
#     print(f"\nTop Tags: {', '.join([t.name for t in result.tags.list[:5]])}")
# if result.objects:
#     print(f"\nObjects: {len(result.objects.list)} detected")
#     annotate_objects(image_file, result.objects.list)
# if result.people:
#     print(f"\nPeople: {len(result.people.list)} detected")
#     annotate_people(image_file, result.people.list)

## Summary

In this lab, you learned how to:
- ✓ Set up and authenticate with Azure AI Vision service
- ✓ Extract captions and tags from images using AI
- ✓ Detect objects and people in images
- ✓ Visualize detection results with bounding boxes
- ✓ Analyze multiple images programmatically

## Next Steps

- Try the **Advanced Image Analysis** notebook to explore more features like:
  - Smart cropping for thumbnails
  - Background removal
  - Dense captioning
  - Reading text in images (OCR)
- Explore the [Azure AI Vision documentation](https://learn.microsoft.com/azure/ai-services/computer-vision/)
- Learn about [best practices for image analysis](https://learn.microsoft.com/azure/ai-services/computer-vision/overview-image-analysis)

## Clean Up

Remember to manage your Azure resources to avoid unexpected charges. You can keep the resources for the next labs or delete them if no longer needed.